# DeepLerning最終課題

松尾研DeepLerning深層学習Springの最終課題のマスターコードである。

## 0.概要

#### 概要
被験者が画像を見ているときの脳波から，その画像がどのカテゴリに属するかを分類するタスク．
- サンプル数: 訓練 118,800 サンプル，検証 59,400 サンプル，テスト 59,400 サンプル
- クラス数: 5
- 入力: 脳波データ（チャンネル数 x 系列長）
- 出力: 対応する画像のクラス
- 評価指標: Top-1 accuracy

#### 修了要件
- ベースラインモデルのbest test accuracyは38.7%となります．**これを超えた提出のみ，修了要件として認めます**．
- ベースラインから改善を加えることで，55%までは性能向上することを運営で確認しています．こちらを 1 つの指標として取り組んでみてください．

#### 注意点
- 学習するモデルについて制限はありませんが，必ず訓練データで学習したモデルで予測してください．
    - 事前学習済みモデルを利用して，訓練データを fine-tuning しても構いません．
    - 埋め込み抽出モデルなど，モデルの一部を訓練しないケースは構いません．
    - 学習を一切せずに，ChatGPT などの基盤モデルを利用することは禁止とします．

In [1]:
#インポート
import os, sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from einops.layers.torch import Rearrange
from einops import repeat
from glob import glob
from termcolor import cprint
from tqdm.notebook import tqdm

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# BaseCord

## 1.Preparation

In [2]:
WORK_DIR = r"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\最終課題"
os.makedirs(WORK_DIR, exist_ok=True)
%cd {WORK_DIR}

C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\最終課題


c:\Users\sudok\AppData\Local\Programs\Python\nova\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## 2.Data

In [9]:
class ThingsEEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str) -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.num_classes = 5
        self.num_subjects = 10

        self.X = np.load(fr"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\eeg.npy")
        self.X = torch.from_numpy(self.X).to(torch.float32)
        self.subject_idxs = np.load(fr"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\subject_idxs.npy")
        self.subject_idxs = torch.from_numpy(self.subject_idxs)

        if split in ["train", "val"]:
            self.y = np.load(fr"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\labels.npy")
            self.y = torch.from_numpy(self.y)

        print(f"EEG: {self.X.shape}, labels: {self.y.shape if hasattr(self, 'y') else None}, subject indices: {self.subject_idxs.shape}")

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

## 3.Code

In [6]:
class ConvBlock(nn.Module):
    def __init__(
        self,
        in_dim,
        out_dim,
        kernel_size: int = 3,
        p_drop: float = 0.1,
    ) -> None:
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        # self.conv2 = nn.Conv1d(out_dim, out_dim, kernel_size) # , padding="same")

        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)

        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X = self.conv0(X) + X  # skip connection
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))

        X = self.conv1(X) + X  # skip connection
        X = F.gelu(self.batchnorm1(X))

        # X = self.conv2(X)
        # X = F.glu(X, dim=-2)

        return self.dropout(X)


class BasicConvClassifier(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 128
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ConvBlock(hid_dim, hid_dim),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, num_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """_summary_
        Args:
            X ( b, c, t ): _description_
        Returns:
            X ( b, num_classes ): _description_
        """
        X = self.blocks(X)

        return self.head(X)

## 4.Traning

In [10]:
# ハイパラ
lr = 0.001
batch_size = 512
epochs = 10
is_cuda = False

# ------------------
#    Dataloader
# ------------------
train_set = ThingsEEGDataset("train") # ThingsMEGDataset("train")
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
val_set = ThingsEEGDataset("val") # ThingsMEGDataset("val")
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=False
)

# ------------------
#       Model
# ------------------
if is_cuda:
    model = BasicConvClassifier(
        train_set.num_classes, train_set.seq_len, train_set.num_channels
    ).to("cuda")
else:
    model = BasicConvClassifier(
        train_set.num_classes, train_set.seq_len, train_set.num_channels
    )

# ------------------
#     Optimizer
# ------------------
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# ------------------
#   Start training
# ------------------
max_val_acc = 0
def accuracy(y_pred, y):
    return (y_pred.argmax(dim=-1) == y).float().mean()

writer = SummaryWriter("tensorboard")

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    model.train()
    for X, y, subject_idxs in tqdm(train_loader, desc="Train"):
        if is_cuda:
            X, y = X.to("cuda"), y.to("cuda")
        else:
            X, y = X.to("cpu"), y.to("cpu")

        y_pred = model(X)

        loss = F.cross_entropy(y_pred, y)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(y_pred, y)
        train_acc.append(acc.item())

    model.eval()
    for X, y, subject_idxs in tqdm(val_loader, desc="Validation"):
        if is_cuda:
            X, y = X.to("cuda"), y.to("cuda")
        else:
            X, y = X.to("cpu"), y.to("cpu")

        with torch.no_grad():
            y_pred = model(X)

        val_loss.append(F.cross_entropy(y_pred, y).item())
        val_acc.append(accuracy(y_pred, y).item())

    print(f"Epoch {epoch+1}/{epochs} | \
        train loss: {np.mean(train_loss):.3f} | \
        train acc: {np.mean(train_acc):.3f} | \
        val loss: {np.mean(val_loss):.3f} | \
        val acc: {np.mean(val_acc):.3f}")

    writer.add_scalar("train_loss", np.mean(train_loss), epoch)
    writer.add_scalar("train_acc", np.mean(train_acc), epoch)
    writer.add_scalar("val_loss", np.mean(val_loss), epoch)
    writer.add_scalar("val_acc", np.mean(val_acc), epoch)

    torch.save(model.state_dict(), "model_last.pt")

    if np.mean(val_acc) > max_val_acc:
        cprint("New best. Saving the model.", "cyan")
        torch.save(model.state_dict(), "model_best.pt")
        max_val_acc = np.mean(val_acc)

EOFError: No data left in file

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tensorboard

## 5.Predict

In [ ]:
# ------------------
#    Dataloader
# ------------------
test_set = ThingsEEGDataset("test")
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False
)

# ------------------
#       Model
# ------------------
model = BasicConvClassifier(
    test_set.num_classes, test_set.seq_len, test_set.num_channels
).to("cuda")
model.load_state_dict(torch.load("model_best.pt", map_location="cuda"))

# ------------------
#  Start evaluation
# ------------------
preds = []
model.eval()
for X, subject_idxs in tqdm(test_loader, desc="Evaluation"):
    preds.append(model(X.to("cuda")).detach().cpu())

preds = torch.cat(preds, dim=0).numpy()
np.save("submission", preds)
print(f"Submission {preds.shape} saved.")

# OriginalCode

## 1.Preparation

In [9]:
class ThingsEEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str) -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.num_classes = 5
        self.num_subjects = 10

        self.X = np.load(fr"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\eeg.npy")
        self.X = torch.from_numpy(self.X).to(torch.float32)
        self.subject_idxs = np.load(fr"CC:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\subject_idxs.npy")
        self.subject_idxs = torch.from_numpy(self.subject_idxs)

        if split in ["train", "val"]:
            self.y = np.load(fr"C:\Users\sudok\OneDrive\ドキュメント\0B-DeepLerning\DeepLerning-FinalSubmission\FinalSubmission\data\{split}\labels.npy")
            self.y = torch.from_numpy(self.y)

        print(f"EEG: {self.X.shape}, labels: {self.y.shape if hasattr(self, 'y') else None}, subject indices: {self.subject_idxs.shape}")

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

In [10]:
train_set = ThingsEEGDataset("train") # ThingsMEGDataset("train")
val_set = ThingsEEGDataset("val") # ThingsMEGDataset("val")

EOFError: No data left in file